# 0) Version Control

Version for Medium article

# 1) Package Import

In [ ]:
import pandas as pd
import time
import random
import os
import openai

# 2) Variable declarations (global)

In [ ]:
# openAI API key
%env OPENAI_API_KEY = sk-YOURVERYCRYPTICOPENAIAPIKEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Define the context for Step1: with this, the job description is fed to chatGPT
# Optimize outputs by playing with this formulation, e.g. the tone of voice
s_context_JobDesc = '''I will give you a job description. I want you to formulate a cover letter that
perfectly aligns with the needs and requirements expressed in this job description.
Keep the tone formal and professional.
The job description is as follows: 
'''

In [ ]:
# Define the context for Step2: with this, the CV is fed to chatGPT to further optimize the cover letter
# Optimize outputs by playing with this formulation, e.g. the focus part
s_context_CV = '''Now that you have formulated the cover letter, optimize it using the the data from my CV that will follow.
Focus on the leadership aspects in my CV.
The CV is as follows: 
'''

In [ ]:
# PASTE YOUR (optimized) plain text CV into this variable
s_CV = '''A) Professional Experience
A1) Finance Director GSA and Eastern Europe since January 2009
ABC Corp.
• Leading FP&A, accounting, treasury, tax, compliance, statutory and corporate matters for marketing & distribution activities
• ....more info  
A2) Sales Director 
ZYX Ltd. October 2006 - October 2008  
• Build and grew the sales for ZYX's fantastic portfolio in 9 Central and Northern European countries via major industrial partners, taking revenue from 0 to the 50 Mio. range.  
• ...more info
A3) General Manager  
Fantastic GmbH / October 2000 - September 2008  
• 01/2006-09/2006, global responsibility for business development activities on new platforms;  prepared entry into new markets, establishing new revenue streams.  
• ... more info  
A4) Team Leader Controlling  
CallCo communications Inc / August 1997 - September 2000  
• From July 1999: Head of Almost Everything with a budget volume in the 100 Mio. EUR range with team of 4 people.  
• ... more info
B) Pre-Professional
B1) Military service
Air Force • October 1986 - July 1987  
B2) EDUCATION  
B2a) Studies of Engineering  
Master at Technische Hochschule • 1993 • 1,9 GPA on 1-6 scale  
B2b) Economic Engineering (Génie Industriel)  
ENSGI (Ecole Nationale Supérieure de Génie Industriel) • France •  
1991 
B2c) A-Levels / German High School Graduation • 1,2 GPA on 1-6 scale  
B2d) US High School Graduation  • 3.8 GPA
C) Skills
C1) IT
- Standard Office Applications:  Expert
- BI-Applications:  Advanced
- PowerBI, Tableau:  Advanced 
- Python (Jupyter / Jupyter Lab) : Good 
- Databases: SQL (MariaDB,  Teradata, Snowflake): Good 
- Data Science: ML / AI Process:  Good
- Privacy and Data Protection:  Advanced
C2) Languages
- German  (mother tongue)
- English (C2 level)  
- French (C2 level)  
- Spanish (B1-B2 level)  
- Dutch (A1 level)'''

# 3) Definitions

## 3a) Class definitions

## 3b) Function Definitions

In [ ]:
def f_cover_letter_generation(s_JobDesc):
    # initiate gpt_input
    gpt_input = [{"role": "system", "content" : 'You are an HR-assistant that helps job candidates'}]

    # Step1: Getting a first cover letter based solely on the job description itself
    ###############################################
    # Joining context and actual job description
    content1 = s_context_JobDesc + s_JobDesc
    ###############################################
    gpt_input.append({"role": "user", "content": content1})
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=gpt_input,
      temperature=1,
      #max_tokens=2500,
    )
    chat_response = completion.choices[0].message.content
    CL_only = chat_response # store return value of step1 of function
    # Next line is VERY important -> it makes sure that the "pure" cover letter is not forgotten
    gpt_input.append({"role": "assistant", "content": chat_response})

    # Step2: optimizing the cover letter with CV data
    ###############################################
    # Joining context and actual CV date
    content2 = s_context_CV + s_CV
    ###############################################
    gpt_input.append({"role": "user", "content": content2})
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=gpt_input,
      temperature=0,
      #max_tokens=2500,
    )
    chat_response = completion.choices[0].message.content
    CL_with_CV = chat_response # store as return value of step2 of function
    gpt_input.append({"role": "assistant", "content": chat_response})   
    
    return (CL_only, CL_with_CV)

# 4) Actual Program

## 4a) Data Import

In [ ]:
# MIND THE NAMING of file AND sheet
xlsx_imp = pd.ExcelFile('LINKEDIN_JOB_POSTINGS_INPUT.xlsx')
df_imp = pd.read_excel(xlsx_imp, 'POST_SELECT')

In [ ]:
# create working dataframe with only the manually selected job postings
# "The 'x' in the column called SELECT marks the spot" :-)
df_w1 = df_imp[df_imp['SELECT'] =='x'].reset_index()
df_w1 = df_w1.drop(['index','idx'], axis=1)
print(len(df_w1))

## 4b) The Core Code: looping thru the postings and have ChatGPT generate optimized cover letters

In [ ]:
l_CL_only = []
l_CL_with_CV = []
l_CL_target = []

In [ ]:
# loop through the rows using iterrows()
for index, row in df_w1.iterrows():
    #############################################################
    print('Generating Cover Letter for role of ' + row['JobTitle']  + ' at ' + row['CompanyName'])
    s_JobDesc = row['JobDescDetails']
    CL_only, CL_with_CV = f_cover_letter_generation(s_JobDesc)
    #############################################################
    l_CL_only.append(CL_only)
    l_CL_with_CV.append(CL_with_CV)
    s_target = 'Cover Letter for role as ' + row['JobTitle']  + ' at ' + row['CompanyName']
    l_CL_target.append(s_target)
    # pause for a random number of seconds
    time.sleep(random.randint(10, 30))

## 4c) Prepare data for export and save to file

In [ ]:
df_w1 = df_w1.drop(['SELECT','SELECT2'], axis=1)
df_w1['CoverLetter_BEFORE_CV'] = l_CL_only
df_w1['CoverLetter_AFTER_CV'] = l_CL_with_CV
df_w1['CL_Target'] = l_CL_target

In [ ]:
df_w1.to_excel("_CHATGPT_COVER_LETTERS.xlsx")